In [99]:
# Obtiene los datos de precipitación par la Cuenca del Valle
# de Méxco a partir de CHIRPS y los promedia espacialmente.

import os

import rioxarray

import pandas as pd
import numpy as np

import geopandas as gpd
import xarray as xr

In [430]:
# Datos.
# viento 925 hPa.

path_ds = "../data/ERA5/"
path_d = "../results/onset/"
path_r = "../results/onset/graficas/"

# Variable, nivel de presión y región.
vars = [ "u", "v", "sst", "olr" ]
levels = [ "925", "200" ]
region = [ "mexico" ]
zones =  (-80, 12.5, -70, 17.5)

# Fechas de onset por precipitación
date_pre = pd.read_csv(path_d + "onset_CHIRPS.csv", index_col = "Año" )
date_pre.index = pd.RangeIndex(date_pre.index[0], date_pre.index[-1] + 1, 
    name = "Año" )
date_pre.iloc[:, 0] = pd.to_datetime( date_pre.iloc[:, 0] )
date_pre.iloc[:, 1] = pd.to_datetime( date_pre.iloc[:, 1] )

# Se convierten las fechas a días julianos.
date_pre["Cuenca - Onset"] = date_pre["Cuenca - Onset"].apply(
    lambda x: pd.to_datetime(x) )
date_pre["Cuenca - Onset"] = date_pre["Cuenca - Onset"].apply(
    lambda x: x.dayofyear )

# Viento zonal.
fname = region[0] + "_" + vars[0] + "_" + levels[0] + "_daily.grib"
ds = xr.open_dataset( path_ds + fname, engine = "cfgrib" )

# Quitamos los 29 de febrero.
ds = ( ds.sel( time = ~( (ds.time.dt.month == 2) & (ds.time.dt.day == 29) ) ) )

# Cantidad de días a promediar.
d = 20

# Seleccionamos la región.
clip = ds.sel( longitude = slice(zones[0], zones[2]), 
    latitude = slice(zones[3], zones[1]) ).copy()

clip = clip.drop_vars(["step", "isobaricInhPa", "valid_time"])

# Media móvil de precipitación centrada a 15 días.
clip = ( clip.rolling(time = d, center = True).mean() )

# Precipitación diaria promedio en toda la cuenca.
df = ( clip.mean(dim = ["latitude", "longitude"]).to_dataframe() )
df.columns = ["Cuenca"]
df["Cuenca"] = - df["Cuenca"]

clip

<xarray.Dataset>
Dimensions:    (time: 14965, latitude: 21, longitude: 41)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01T06:00:00 ... 2021-12-31T06:00:00
  * latitude   (latitude) float64 17.5 17.25 17.0 16.75 ... 13.0 12.75 12.5
  * longitude  (longitude) float64 -80.0 -79.75 -79.5 ... -70.5 -70.25 -70.0
Data variables:
    u          (time, latitude, longitude) float64 nan nan nan ... nan nan nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2022-04-18T19:54:42 GRIB to CDM+CF via cfgrib-0....

In [397]:
def onset( vars = [12.5, 7] , m_i = 5, df_mean_ind = df.copy() ):

    umbral = vars[0]
    n = int( vars[1] )

    c = "Cuenca"

    # Nombre de las columnas dummy.
    proc = [c]
    for i in range( 0, n ): proc.append(c + "_" + str(i))

    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    for i, p in enumerate(proc[1:]):
        df_mean_ind[p] = df_mean_ind[proc[0]].shift(i)
        df_mean_ind[p].clip( upper = umbral, inplace = True )
        df_mean_ind[p] = df_mean_ind[p].where( df_mean_ind[p] == umbral, 0 )
        df_mean_ind[p] = df_mean_ind[p].where( df_mean_ind[p] < umbral, 1 )

    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df_mean_ind[ind] = 0
    for i in range( n ):
        df_mean_ind[ind] += df_mean_ind[proc[i + 1]]
    df_mean_ind[ind] = df_mean_ind[ind].where(
        df_mean_ind[ind] == n, 0 )
    df_mean_ind[ind] = df_mean_ind[ind].where(
        df_mean_ind[ind] < n, 1 )
    df_mean_ind[ind] = df_mean_ind[ind].where( df_mean_ind.index.month.isin(
        range(m_i, 12) ), 0 )
    df_mean_ind.drop(proc[1:], axis = 1, inplace = True)

    # Crea una tabla con los valores anuales.
    date = pd.DataFrame()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df_mean_ind[ (df_mean_ind.index.year == j)
            & df_mean_ind.index.month.isin(range(m_i, 12)) ]

        # Fechas.
        date = date.append( [ df_year.iloc[:, 1].idxmax() ] )

    # Damos formato a las columnas y al índice.
    date.index = years
    # Recorremos al inicio de la racha de días de precipitación.
    date.iloc[:, 0] = date.iloc[:, 0] - np.timedelta64(n, 'D')

    # Correlación entre índice de precipitación y de viento.

    # Se convierten las fechas a días julianos.
    date.iloc[:, 0] = date.iloc[:, 0].apply(
        lambda x: pd.to_datetime(x))
    date.iloc[:, 0] = date.iloc[:, 0].apply(
        lambda x: x.dayofyear)

    # Correlaciones de las series.
    corr = date.iloc[:, 0].corr( date_pre["Cuenca - Onset"] )

    # Liberamos memoria.
    del date
    del df_mean_ind

    return corr

In [433]:
def onset( vars = [12.5, 7] , m_i = 5, df = df ):

    umbral = vars[0]
    n = int( vars[1] )

    c = "Cuenca"

    # Nombre de las columnas dummy.
    proc = [c]
    for i in range( 0, n ): proc.append(c + "_" + str(i))

    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    for i, p in enumerate(proc[1:]):
        df[p] = df[proc[0]].shift(i)
        df[p].clip( upper = umbral, inplace = True )
        df[p] = df[p].where( df[p] == umbral, 0 )
        df[p] = df[p].where( df[p] < umbral, 1 )

    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df[ind] = 0
    for i in range( n ):
        df[ind] += df[proc[i + 1]]
    df[ind] = df[ind].where( df[ind] == n, 0 )
    df[ind] = df[ind].where( df[ind] < n, 1 )
    df[ind] = df[ind].where( df.index.month.isin( range(m_i, 12) ), 0 )
    df.drop(proc[1:], axis = 1, inplace = True)

    # Crea una tabla con los valores anuales.
    date = date_pre["Cuenca - Onset"].copy()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df[ (df.index.year == j)
            & df.index.month.isin(range(m_i, 12)) ]

        # Fechas.
        date[j] = df_year.iloc[:, 1].idxmax()

    # Damos formato a las columnas y al índice.
    date.index = years
    # Recorremos al inicio de la racha de días de precipitación.
    date = date - np.timedelta64(n, 'D')

    # Correlación entre índice de precipitación y de viento.

    # Se convierten las fechas a días julianos.
    date = date.apply( lambda x: pd.to_datetime(x) )
    date = date.apply( lambda x: x.dayofyear )

    # Correlaciones de las series.
    corr = date.corr( date_pre["Cuenca - Onset"] )

    # Liberamos memoria.
    del date

    return corr

In [426]:
onset( c[0, 0] )

-0.046708741740563706

In [436]:
#a = np.arange(1, 20)
#b = np.arange(10, 20, 0.25)

a = np.arange(1, 20)
b = np.arange(5, 20, 0.5)

c = np.stack( np.meshgrid( b, a ), axis = 2 )

res = np.ndarray(shape = c.shape[0:2])

for i in range(c.shape[0]):
    for j in range(c.shape[1]):
        res[i, j] = onset( c[i, j] )

print( res.max() )
print( c[np.argmax(res) // res.shape[1], np.argmax(res) % res.shape[1]] )

0.2188462767188315
[12.5  3. ]
